<a href="https://colab.research.google.com/github/LavanyaSNS/Automation-AI-Model/blob/main/Copy_of_Copy2_of_Final_AI_model_Task_for_BMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install yt-dlp whisper transformers accelerate torch pytesseract pydub webrtcvad easyocr groq

# Download YOLOv3 weights and configuration files
!wget https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg -O yolov3.cfg
!wget https://github.com/pjreddie/darknet/blob/master/yolov3.weights -O yolov3.weights

# Install ffmpeg
!apt-get install -y ffmpeg

# Check ffmpeg version
!ffmpeg -version

!pip install git+https://github.com/openai/whisper.git

# Task 1: Download and Validate YouTube Video and Audio

1. Video Information Retrieval: Getting the title of the video from the URL.
2. Video Downloading: Downloading the video in a specified format.
3. Video Quality Checking: Verifying the resolution of the video.
4. Audio Extraction: Extracting audio from the downloaded video.
5. Audio Quality Checks: Ensuring the audio meets quality standards (bitrate, channels, duration).
6. Tone and Noise Checks: Analyzing the audio for pitch and background noise.
7. Voice Presence Detection: Using VAD to check if human speech is present in the audio.
8. Audio Transcription: Converting the audio to text using Whisper.

In [ ]:
import os
import yt_dlp
import subprocess
import whisper
from pydub import AudioSegment
import librosa
import webrtcvad
import numpy as np
import wave
import contextlib

def get_video_info(video_url):
    ydl_opts = {'skip_download': True}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(video_url, download=False)
        title = info.get('title', 'No title found')
        return title

def download_video(video_url, output_path="video.mp4.mkv"):
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'outtmpl': output_path,
        'merge_output_format': 'mkv',
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])
        return output_path
    except Exception as e:
        print(f"Error downloading video: {e}")
        return None

def check_video_quality(video_path):
    command = [
        'ffprobe',
        '-v', 'error',
        '-select_streams', 'v:0',
        '-show_entries', 'stream=width,height',
        '-of', 'csv=p=0',
        video_path
    ]
    try:
        output = subprocess.check_output(command).decode().strip()
        width, height = map(int, output.split(','))

        if height < 480:
            return False, f"Video quality issue: The video resolution is {width}x{height}, but it must be at least 480p."
        return True, f"Video is valid: {width}x{height} resolution."

    except subprocess.CalledProcessError as e:
        return False, f"Error checking video quality: {e}"

def extract_audio(video_path, audio_output_path="audio.mp3"):
    command = [
        'ffmpeg',
        '-i', video_path,
        '-q:a', '0',
        '-map', 'a',
        audio_output_path
    ]
    try:
        subprocess.run(command, check=True, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
        return audio_output_path
    except subprocess.CalledProcessError as e:
        print(f"Error extracting audio: {e.stderr.decode()}")  # Print error message from ffmpeg
        return None


# Audio Quality, Tone, Background Noise Checks

def check_audio_quality(audio_path):
    audio = AudioSegment.from_mp3(audio_path)
    bitrate = audio.frame_rate
    channels = audio.channels
    duration = len(audio) / 1000  # Duration in seconds

    if bitrate < 16000:  # Sample rate should be at least 16kHz
        return False, "Audio quality too low (bitrate < 16kHz)."
    if channels != 1:
        # Convert to mono if it has multiple channels
        audio = audio.set_channels(1)
        audio.export(audio_path, format='mp3')  # Overwrite the original audio
        channels = 1  # Update the channels count

    if duration < 240:  # Check if the audio is less than 4 minutes (240 seconds)
        return False, "Audio file too short (less than 4 minutes)."

    return True, "Audio quality is acceptable."



def check_audio_tone(audio_path):
    audio_data, sr = librosa.load(audio_path)
    pitches, magnitudes = librosa.piptrack(y=audio_data, sr=sr)

    # Calculate the mean pitch
    pitch_values = [pitches[i][pitches[i] > 0] for i in range(len(pitches))]
    pitch_mean = [np.mean(pitch) if len(pitch) > 0 else 0 for pitch in pitch_values]

    if np.mean(pitch_mean) < 75:  # Too low, possibly not human speech
        return False, "Audio tone may be too low for valid speech."
    return True, "Audio tone is acceptable."


def check_background_noise(audio_path):
    audio_data, sr = librosa.load(audio_path)
    energy = np.sum(audio_data ** 2) / len(audio_data)
    noise_threshold = np.percentile(np.abs(audio_data), 5)
    noise_energy = np.mean(np.abs(audio_data)[np.abs(audio_data) < noise_threshold] ** 2)

    snr = 10 * np.log10(energy / noise_energy)

    if snr < 15:  # Minimum SNR of 15dB
        return False, "Too much background noise."
    return True, "Background noise level is acceptable."


def convert_to_pcm(audio_path):
    """
    Convert audio to 16-bit mono PCM with a sample rate of 16kHz.
    """
    sound = AudioSegment.from_file(audio_path)
    sound = sound.set_channels(1)  # Convert to mono
    sound = sound.set_frame_rate(16000)  # Set sample rate to 16kHz
    pcm_path = "converted_audio.wav"
    sound.export(pcm_path, format="wav")
    return pcm_path


def check_voice_presence(audio_path):
    """
    Check if the audio contains human speech using webrtcvad.
    """
    pcm_audio_path = convert_to_pcm(audio_path)

    vad = webrtcvad.Vad()
    vad.set_mode(1)  # Mode 1: more aggressive detection of speech

    with contextlib.closing(wave.open(pcm_audio_path, 'rb')) as wf:
        sample_rate = wf.getframerate()
        assert sample_rate == 16000, "Sample rate must be 16000 Hz"
        pcm_data = wf.readframes(wf.getnframes())

        frame_duration = 30  # Frame size: 30ms
        frame_size = int(sample_rate * frame_duration / 1000)

        for i in range(0, len(pcm_data), frame_size * 2):  # 2 bytes per sample (16-bit audio)
            frame = pcm_data[i:i + frame_size * 2]
            if vad.is_speech(frame, sample_rate):
                return True, "Voice detected in the audio."

    return False, "No voice detected in the audio."


def transcribe_audio(audio_path):
    model = whisper.load_model("small")
    result = model.transcribe(audio_path)

    transcription_text_file = "/content/transcription.txt"
    transcription_text = result['text']
    with open(transcription_text_file, 'w') as f:
        f.write(transcription_text)

    print("Transcription saved to transcription.txt")
    return transcription_text_file

# Example Usage
video_url = 'https://youtu.be/54keG0E81xQ?si=2saiQTvV2gtWcDy2'
title = get_video_info(video_url)
video_file = download_video(video_url)

if video_file:
    print(f"Video downloaded successfully: {video_file}")
    print(f"Video title: {title}")

    quality_check, quality_report = check_video_quality(video_file)
    print(quality_report)

    if quality_check:  # Proceed to extract audio only if the video quality is satisfied
        audio_file = extract_audio(video_file)
        if audio_file:  # Check if audio was extracted successfully
            print(f"Audio extracted successfully: {audio_file}")

            audio_checks = [
                check_audio_quality(audio_file),  # Returns (True/False, "message")
                check_audio_tone(audio_file),     # Returns (True/False, "message")
                check_background_noise(audio_file),# Returns (True/False, "message")
                check_voice_presence(audio_file)  # Returns (True/False, "message")
            ]

            # Check if all conditions are satisfied
            if all(check[0] for check in audio_checks):  # Each `check` is a tuple now
                transcription_file = transcribe_audio(audio_file)
                print(f"Transcription saved to: {transcription_file}")
            else:
                # If any condition fails, print the corresponding message
                for check in audio_checks:
                    if not check[0]:  # If the check failed
                        print(check[1])  # Print the associated message

        else:
            print("Audio extraction failed.")
    else:
        print("Audio extraction skipped due to low video quality.")
else:
    print("Failed to download the video.")



# Frame Checking

In [ ]:
import cv2
import os

def extract_images_from_video(video_path, output_dir="frames", interval=10):
    """
    Extract frames from video every 'interval' seconds.
    :param video_path: Path to the video file.
    :param output_dir: Directory to save the extracted images.
    :param interval: Time interval between frames to extract (in seconds).
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)  # Frames per second
    frame_interval = int(fps * interval)

    count = 0
    success = True
    while success:
        success, frame = cap.read()
        if count % frame_interval == 0 and success:
            # Use zero-padding to ensure correct sorting
            img_path = os.path.join(output_dir, f"frame_{count:05d}.jpg")
            cv2.imwrite(img_path, frame)
        count += 1

    cap.release()
    print(f"Images extracted to {output_dir}")

# Example usage
extract_images_from_video('/content/video.mp4.mkv', interval=15)

Images extracted to frames


# Define Functions for Text Extraction and Uniqueness Check

In [ ]:
import os
import cv2
import easyocr
import whisper
from skimage.metrics import structural_similarity as ssim
import numpy as np

def is_unique_image(img_path, unique_images, similarity_threshold=0.95):
    """
    Check if the image is unique compared to already processed unique images.
    :param img_path: Path to the image file.
    :param unique_images: List of already processed unique images.
    :param similarity_threshold: Similarity threshold for uniqueness.
    :return: True if the image is unique, False otherwise.
    """
    img = cv2.imread(img_path)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    for unique_img in unique_images:
        unique_img_gray = cv2.cvtColor(unique_img, cv2.COLOR_BGR2GRAY)
        # Calculate the Structural Similarity Index (SSI)
        score = ssim(img_gray, unique_img_gray)
        if score >= similarity_threshold:
            return False  # Image is similar to an already processed unique image

    return True  # Image is unique

'''def extract_text_from_image(img_path):
    """
    Extract text from an image using EasyOCR.
    :param img_path: Path to the image file.
    :return: Extracted text.
    """
    reader = easyocr.Reader(['en'])
    results = reader.readtext(img_path)
    extracted_text = " ".join([text[1] for text in results])  # Concatenate all detected text
    return extracted_text'''

def extract_text_from_image(img_path):
    """
    Extract text from an image using EasyOCR. Skip if no text is detected.
    :param img_path: Path to the image file.
    :return: Extracted text or None if no text is detected.
    """
    reader = easyocr.Reader(['en'])
    results = reader.readtext(img_path)

    if not results:  # No text detected
        print(f"No text detected in image: {img_path}")
        return None

    extracted_text = " ".join([text[1] for text in results])  # Concatenate all detected text
    return extracted_text

def analyze_frames(frames_dir, similarity_threshold=0.95):
    """
    Analyze frames in a directory to extract text from unique images.
    :param frames_dir: Directory containing the frames.
    :param similarity_threshold: Threshold for image similarity.
    :return: Dictionary of extracted texts from unique images.
    """
    unique_images = []
    extracted_texts = {}

    for image_file in os.listdir(frames_dir):
        img_path = os.path.join(frames_dir, image_file)
        if is_unique_image(img_path, unique_images, similarity_threshold):
            # If unique, read the image and extract text
            extracted_text = extract_text_from_image(img_path)
            extracted_texts[image_file] = extracted_text
            # Store the image for future uniqueness checks
            unique_images.append(cv2.imread(img_path))

    return extracted_texts

def transcribe_audio(audio_path):
    """
    Transcribe audio using Whisper model.
    :param audio_path: Path to the audio file.
    :return: Transcribed text.
    """
    model = whisper.load_model("small")
    result = model.transcribe(audio_path)
    return result['text']

# Paths
frames_directory = '/content/frames'  # Path to the frames directory
audio_file_path = '/content/audio.mp3'  # Path to the audio file

# Extract text from frames and transcribe audio
extracted_texts = analyze_frames(frames_directory)
audio_text = transcribe_audio(audio_file_path)

# Combine texts
combined_text = "Extracted Texts from Frames:\n"
for img_name, text in extracted_texts.items():
    combined_text += f"Extracted Text from {img_name}:\n{text}\n\n"

combined_text += "Transcribed Audio Text:\n"
combined_text += audio_text

# Save to a text file
output_file_path = '/content/extracted_texts_combined.txt'
with open(output_file_path, 'w') as file:
    file.write(combined_text)

print(f"Combined extracted texts saved to {output_file_path}")

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

No text detected in image: /content/frames/frame_04950.jpg
No text detected in image: /content/frames/frame_00900.jpg


No text detected in image: /content/frames/frame_03600.jpg


No text detected in image: /content/frames/frame_01350.jpg


...

No text detected in image: /content/frames/frame_00450.jpg


No text detected in image: /content/frames/frame_06300.jpg


No text detected in image: /content/frames/frame_05850.jpg


100%|███████████████████████████████████████| 461M/461M [00:07<00:00, 61.1MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Combined extracted texts saved to /content/extracted_texts_combined.txt


# Text Verification

In [ ]:
import requests
import json

# Set the TogetherAI API key
api_key = "0b79e2f0ddb16654bc98df9f828e0474d53c7d00eac41328abf06bd4858d14bb"

# Load the transcribed text from the file
with open("/content/extracted_texts_combined.txt", "r") as file:
    transcript = file.read()

# Set the video title
title = "CONVENTIONAL SCREW"

# Create a more detailed and structured prompt for verification task
prompt = f"""
You are tasked with analyzing the transcription of a business model canvas (BMC) presentation video.
Please perform the following checks and provide the results in JSON format:

1. **Names:** Extract the names of students mentioned when they introduce themselves.
2. **Introduction:** Check if there is a formal introduction at the beginning of the transcription. Indicate 'Yes' if an introduction is present and 'No' if not.
3. **Relevance:** Verify if the content of the transcription is relevant to the video title '{title}'. Answer 'Yes' or 'No' and explain briefly why the content is or is not relevant.
4. **BMC Topics:** Confirm whether the transcription covers all 9 BMC topics: key partners, key activities, value propositions, customer relationships, customer segments, key resources, channels, cost structure, and revenue streams. Answer 'Yes' or 'No' and list the missing topics if any.

Transcription to analyze:
{transcript}

Please provide your analysis in the following JSON format:

{{
    "Names": ["List of names"],
    "Introduction": "Yes/No",
    "Relevance": {{
        "IsRelevant": "Yes/No",
        "Reason": "Brief explanation of relevance"
    }},
    "BMC Topics": {{
        "AllTopicsCovered": "Yes/No",
        "MissingTopics": ["List of missing topics, if any"]
    }}
}}
"""

# API endpoint
url = "https://api.together.xyz/v1/chat/completions"

# Headers
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# Request payload
data = {
    "model": "meta-llama/Llama-Vision-Free",
    "messages": [{"role": "user", "content": prompt}]
}

# Send the request to TogetherAI API
response = requests.post(url, headers=headers, data=json.dumps(data))

# Check if the request was successful
if response.status_code == 200:
    # Parse the response
    response_content = response.json()
    # Extract the relevant message from the response
    message_content = response_content["choices"][0]["message"]["content"]

    # Print the raw response
    print("Raw Response:", message_content)

    # Try to parse it as JSON
    try:
        response_data = json.loads(message_content)  # Load the response as JSON
        print("Parsed Response:", json.dumps(response_data, indent=2))  # Pretty print the parsed JSON
    except json.JSONDecodeError as e:
        print("Failed to decode JSON response:", e)
else:
    print(f"Request failed with status code {response.status_code}: {response.text}")


Raw Response: Here is the analysis in JSON format:

```json
{
    "Names": ["Steira", "Sandesh", "Prasimha Bairdhi", "Santakumar"],
    "Introduction": "Yes",
    "Relevance": {
        "IsRelevant": "No",
        "Reason": "The content of the transcription is about a project to design a conventional screw with improved functionality and rust prevention, which is not directly related to the video title 'CONVENTIONAL SCREW'."
    },
    "BMC Topics": {
        "AllTopicsCovered": "No",
        "MissingTopics": ["Channels", "Cost structure", "Revenue streams"]
    }
}
```

In the Names section, I extracted the names of the students mentioned in the transcription.

For the Introduction section, I marked it as "Yes" because there is a brief introduction at the beginning of the transcription.

For the Relevance section, I marked it as "No" because the content of the transcription is about a project to design a conventional screw, which is not directly related to the video title 'CONVENTIONA